## Vector 기반의 유사도 검색 후 RAG로 공고 요약을 해보자!

In [2]:
import json
import numpy as np
import faiss

from sentence_transformers import SentenceTransformer
from getpass import getpass
from tqdm import tqdm

from utils.get_text import get_hwp_text, get_pdf_text, get_image_text # pdf, hwp, png에서 텍스트를 추출하는 사용자 지정 함수 

#### model 구현 준비
- OpenAI 객체 생성
- 파일 텍스트 추출
- model이 참고할 기본 템플릿 작성

In [2]:
# # 개인의 API key를 입력
# MY_API_KEY = getpass("OpenAI API Key")

# #OpenAI 객체 생성
# client = OpenAI(api_key=MY_API_KEY)

- 문장을 벡터로 임베딩할 모델로드

In [3]:
# Sentence-Transformers 라이브러리에서 모델 로드
# "intfloat/multilingual-e5-large" 모델은 다양한 언어에서 문장 임베딩을 생성할 수 있는 모델
model = SentenceTransformer("intfloat/multilingual-e5-large")

#### 데이터 임베딩 진행
- 데이터 로드 및 각각의 공고문 텍스트 가져오기
- 텍스트를 Vector화

In [4]:
biz_embeddings = [] # 벡터를 저장할 리스트
biz_names = [] # 공고명을 저장할 리스트

In [7]:
with open('data/bizinfo_2.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [8]:
data

{'금융': [{'지원사업 공고명': '[인천] 2025년 소공인 지원 특례보증 사업 공고(소공인 경영안정 및 성장지원)',
   '소관부처·지자체': '인천광역시',
   '사업수행기관': '인천신용보증재단',
   '신청기간': '예산 소진시까지',
   '공고파일명': '금융/[인천] 2025년 소공인 지원 특례보증 사업 공고(소공인 경영안정 및 성장지원).hwp'},
  {'지원사업 공고명': '2025년 전문 투자기관 연계형 투자유치 지원사업 지원기업 모집 공고',
   '소관부처·지자체': '보건복지부',
   '사업수행기관': '한국보건산업진흥원',
   '신청기간': '2025.03.17 ~ 2025.04.07',
   '공고파일명': '금융/2025년 전문 투자기관 연계형 투자유치 지원사업 지원기업 모집 공고.pdf'},
  {'지원사업 공고명': '[광주] 2025년 중소사업장 가족친화경영 지원사업 공고(가족친화경영 문화 조성 및 활성화)',
   '소관부처·지자체': '광주광역시',
   '사업수행기관': '광주광역시일가정양립지원본부',
   '신청기간': '2025.03.26 ~ 2025.04.02',
   '공고파일명': '금융/[광주] 2025년 중소사업장 가족친화경영 지원사업 공고(가족친화경영 문화 조성 및 활성화).hwp'},
  {'지원사업 공고명': '[경기] 화성시 2025년 창업투자펀드 투자상담 창구 운영 공고',
   '소관부처·지자체': '경기도',
   '사업수행기관': '화성산업진흥원',
   '신청기간': '2025.03.10 ~ 2025.11.10',
   '공고파일명': '금융/[경기] 화성시 2025년 창업투자펀드 투자상담 창구 운영 공고.pdf'},
  {'지원사업 공고명': '2025년 해양수산 투자 프로그램 패키지 지원사업 공고',
   '소관부처·지자체': '해양수산부',
   '사업수행기관': '해양수산과학기술진흥원',
   '신청기간': '상시 접수',
   '공고파일명': '금융/2025년 해양수산

In [6]:
# ---------------------------------------------------------------------------------------------------------

In [ ]:
for item in tqdm(data['금융']):
    file_path = item["공고파일명"]

    # 지원사업 텍스트 추출 (PDF/HWP 처리 함수 필요)
    if file_path.endswith(".pdf"):
        full_text = get_pdf_text(file_path)
    elif file_path.endswith(".hwp"):
        full_text = get_hwp_text(file_path)
    elif file_path.endswith(".png"):
        full_text = get_image_text(file_path)

    # 벡터화 및 정규화
    vector = model.encode(full_text)
    vector = vector / np.linalg.norm(vector)  # 정규화 (선택적)

    # 'vector' 키 추가
    item["vector"] = vector.tolist()  # JSON 저장을 위해 리스트로 변환


# 변환된 데이터를 JSON 파일로 저장
with open("data/data_with_vectors_2.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("✅ 벡터가 추가된 JSON 파일 저장 완료!")

100%|██████████| 163/163 [08:24<00:00,  3.10s/it]


✅ 벡터가 추가된 JSON 파일 저장 완료!


#### 유사도 기반 공고 추천

In [8]:
for category, items in data.items():
    for item in items:
        if "vector" in item:  # 벡터 정보가 있는 경우만 추가
            biz_embeddings.append(np.array(item["vector"]))  # 리스트 → numpy 배열 변환
            biz_names.append(item["지원사업 공고명"])

# FAISS 인덱스 생성
biz_dimension = len(biz_embeddings[0])  # 벡터 차원 수
biz_index = faiss.IndexFlatL2(biz_dimension)  # L2 거리 기반 인덱스
biz_index.add(np.array(biz_embeddings))  # 벡터 추가

# ✅ 사용자 입력을 벡터화
query = '''**사업자 유형** - 개인사업자, 법인사업자, 기존사업자
:확인_표시가_있는_투표함: **지원업종** - 제조업, 정보통신업, 건설업, 운수업, 서비스업
지역 충북
신청기간 현재 진행중인 것.
대표자생년월일 x
성별 x
매출 x
종업원수 x'''
query_vector = model.encode(query).reshape(1, -1)
query_vector = query_vector / np.linalg.norm(query_vector)  # 정규화

# ✅ FAISS를 이용한 유사도 검색
top_k = 5  # 검색할 개수
distances, indices = biz_index.search(query_vector, top_k)

# ✅ 검색 결과 출력
print("🔍 검색 결과:")
for i, idx in enumerate(indices[0]):
    print(f"{i+1}. 지원사업명: {biz_names[idx]} (유사도 점수: {distances[0][i]:.4f})")

🔍 검색 결과:
1. 지원사업명: [충북] 2025년 사회적기업 육성자금 이차보전금 지원계획 공고 (유사도 점수: 0.2583)
2. 지원사업명: [충북] 2025년 사회적기업 육성자금 이차보전금 지원계획 공고 (유사도 점수: 0.2583)
3. 지원사업명: 2025년 소상공인 대환대출 지원계획 공고 (유사도 점수: 0.2632)
4. 지원사업명: 2025년 소상공인 대환대출 지원계획 공고 (유사도 점수: 0.2632)
5. 지원사업명: 2025년 중소벤처기업부 소관 소상공인 정책자금 융자계획 공고 (유사도 점수: 0.2682)
